*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [1]:
# import libraries
# try:
#   # %tensorflow_version only exists in Colab.
#   !pip install tf-nightly
# except Exception:
#   pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
# !pip install tensorflow-datasets
# import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

/home/lijo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lijo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lijo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lijo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWa

1.14.0


In [4]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2020-12-31 18:07:09--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 2606:4700:8ddb:45a8:d3de:26:652f:76d6, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|2606:4700:8ddb:45a8:d3de:26:652f:76d6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1        [            <=>     ] 349.84K   105KB/s    in 3.3s    

2020-12-31 18:07:13 (105 KB/s) - ‘train-data.tsv.1’ saved [358233]

--2020-12-31 18:07:13--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 2606:4700:8ddb:45a8:d3de:26:652f:76d6, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|2606:4700:8ddb:45a8:d3de:26:652f:76d6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [t

In [87]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding

In [37]:
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'text'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'text'])

In [95]:
vocab_size = 50000

X_train = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in train_df['text']]
X_test = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in test_df['text']]

In [96]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(['spam', 'ham'])
y_train = le.transform(train_df['label'])
y_test = le.transform(test_df['label'])
le.inverse_transform([1])[0]

'spam'

In [97]:
max_length = 20
X_train = pad_sequences(X_train, maxlen=max_length, padding='pre')
X_test = pad_sequences(X_test, maxlen=max_length, padding='pre')

In [98]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

Train on 3343 samples, validate on 836 samples
Epoch 1/10
3343/3343 [==============================] - 3s 989us/sample - loss: 0.2400 - acc: 0.9121 - val_loss: 0.1239 - val_acc: 0.9761
Epoch 2/10
3343/3343 [==============================] - 2s 628us/sample - loss: 0.0824 - acc: 0.9853 - val_loss: 0.0882 - val_acc: 0.9833
Epoch 3/10
3343/3343 [==============================] - 2s 653us/sample - loss: 0.0468 - acc: 0.9886 - val_loss: 0.0703 - val_acc: 0.9868
Epoch 4/10
3343/3343 [==============================] - 2s 642us/sample - loss: 0.0279 - acc: 0.9916 - val_loss: 0.0557 - val_acc: 0.9916
Epoch 5/10
3343/3343 [==============================] - 2s 644us/sample - loss: 0.0176 - acc: 0.9943 - val_loss: 0.0692 - val_acc: 0.9892
Epoch 6/10
3343/3343 [==============================] - 2s 647us/sample - loss: 0.0126 - acc: 0.9967 - val_loss: 0.0714 - val_acc: 0.9880
Epoch 7/10
3343/3343 [==============================] - 2s 665us/sample - loss: 0.0088 - acc: 0.9976 - val_loss: 0.0760 - val

In [41]:
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())

history = model.fit(X_train, y_train, epochs=20, verbose=0)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 8)             40000     
_________________________________________________________________
flatten_3 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 161       
Total params: 40,161
Trainable params: 40,161
Non-trainable params: 0
_________________________________________________________________
None


In [91]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=1)
print('Training Accuracy is {}'.format(accuracy*100))

4179/4179 [==============================] - 0s 101us/sample - loss: 0.0237 - acc: 0.9950
Training Accuracy is 99.49748516082764


In [92]:
loss, accuracy = model.evaluate(X_test,y_test)
print('Testing Accuracy is {} '.format(accuracy*100))

1392/1392 [==============================] - 0s 114us/sample - loss: 0.0929 - acc: 0.9828
Testing Accuracy is 98.2758641242981 


In [78]:
test_phrase = "i dont want to go. can we try it a different day? available sat"
x = [one_hot(test_phrase, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ')]
x = pad_sequences(x, maxlen=max_length, padding='pre')
x
int(np.round(model.predict(x)))
le.inverse_transform([int(np.round(model.predict(x)))])[0]

'ham'

In [84]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'text'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'text'])

vocab_size = 1000

X_train = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in train_df['text']]
X_test = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in test_df['text']]

le = preprocessing.LabelEncoder()
le.fit(['spam', 'ham'])
y_train = le.transform(train_df['label'])
y_test = le.transform(test_df['label'])

max_length = 25
X_train = pad_sequences(X_train, maxlen=max_length, padding='pre')
X_test = pad_sequences(X_test, maxlen=max_length, padding='pre')

model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=20, verbose=0)    

In [93]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    x = [one_hot(test_phrase, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ')]
    x = pad_sequences(x, maxlen=max_length, padding='pre')

    prediction = [model.predict(x), le.inverse_transform([int(np.round(model.predict(x)))])[0]]
    return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[array([[3.206029e-05]], dtype=float32), 'ham']


In [99]:
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  
  for msg,ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        print(prediction[1], ans)

test_predictions()

ham ham
ham spam
ham ham
ham spam
ham spam
ham ham
ham ham


In [74]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You haven't passed yet. Keep trying.
